In [2]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

## Default Values

In [5]:
# Define path to files
path = "D:/Python for Finance/Stocks/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

## Get Stock File Names in a List

In [6]:
# listdir returns all files in the directory and isfile will return true
# if it is a file and then we store its name in our list named files
files = [x for x in listdir(path) if isfile(join(path, x))]

# Remove extension from file names
# Splitext splits the file name into 2 parts being the name and extension
# We say get all file names and then store just the name in our list named files
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['A',
 'AA',
 'AAL',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAWW',
 'ABBV',
 'ABC',
 'ABCB',
 'ABEO',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABT',
 'ABTX',
 'AC',
 'ACA',
 'ACAD',
 'ACBI',
 'ACC',
 'ACCO',
 'ACER',
 'ACGL',
 'ACHC',
 'ACHV',
 'ACIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACTG',
 'ACU',
 'ACY',
 'ADBE',
 'ADC',
 'ADES',
 'ADI',
 'ADM',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADNT',
 'ADP',
 'ADS',
 'ADSK',
 'ADT',
 'ADTN',
 'ADUS',
 'ADVM',
 'ADXS',
 'AE',
 'AEE',
 'AEHR',
 'AEIS',
 'AEL',
 'AEMD',
 'AEO',
 'AEP',
 'AERI',
 'AES',
 'AEY',
 'AFG',
 'AFI',
 'AFIN',
 'AFL',
 'AGCO',
 'AGE',
 'AGEN',
 'AGFS',
 'AGIO',
 'AGLE',
 'AGM',
 'AGNC',
 'AGO',
 'AGR',
 'AGRX',
 'AGS',
 'AGTC',
 'AGX',
 'AGYS',
 'AHH',
 'AHT',
 'AIG',
 'AIMC',
 'AIN',
 'AINC',
 'AIR',
 'AIRG',
 'AIRI',
 'AIRT',
 'AIT',
 'AIV',
 'AIZ',
 'AJG',
 'AJRD',
 'AJX',
 'AKAM',
 'AKBA',
 'AKR',
 'AKTS',
 'AL',
 'ALB',
 'ALBO',
 'ALCO',
 'ALDX',
 'ALE'

## Create a Dataframe from our List

In [7]:
stock_df = pd.DataFrame(tickers,columns=['Ticker'])
stock_df

,Ticker
0,A
1,AA
2,AAL
3,AAME
4,AAN
...,...
3046,ZUMZ
3047,ZUO
3048,ZVO
3049,ZYNE


## Function that Returns a Dataframe from a CSV

In [8]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(path + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

## Function that Saves Dataframe to CSV

In [9]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(path + ticker + '.csv')

## Delete Unnamed Columns in CSV Files

In [10]:
def delete_unnamed_cols(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

## Add Daily Return to Dataframe

In [11]:
# We calculate a percentage rate of return for each day to compare investments.
# Simple Rate of Return = (End Price - Beginning Price) / Beginning Price OR (EP / BP) - 1

# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    # Save data to a CSV file
    df.to_csv(path + ticker + '.csv')
    return df  

## Returns Return on Investment over Time

In [44]:
# Return on Investment is the return you received from your investment
# This amount does not include your initial investment
# If you invest 100 and have 200 after 5 years
# ROI = End Value (200) - Initial Value (100) / Inital Value = 1
# Your new total is Inital Investment + 1 * Inital Investment = 200

def get_roi_defined_time(df):
    # Set as a datetime
    df['Date'] = pd.to_datetime(df['Date'])
    start_val = df[df['Date'] == S_DATE_STR]['Adj Close'][0]
    print("Initial Price :", start_val)
    
    # ----- I CHANGED THIS AFTER THE VIDEO -----
    
    end_val = df[df['Date'] == E_DATE_STR]['Adj Close']
    print(end_val.item())
    print("Final Price :", end_val.item())
    
    # ----- END OF VIDEO CHANGES -----
    
    # Calculate return on investment
    roi = (end_val - start_val) / start_val

    # Return the total return between 2 dates
    return roi

## Get Coefficient of Variation

In [13]:
# Receives the dataframe with the Adj Close data and returns the coefficient of variation
def get_cov(stock_df):
    mean = stock_df['Adj Close'].mean()
    sd = stock_df['Adj Close'].std()
    cov = sd / mean
    return cov

## Test Functions

In [19]:
# Create a backup for all original stock data

# Get our 1st ticker
tickers[0]

# Get a dataframe for that ticker
stock_a = get_df_from_csv(tickers[0])
stock_a

# Add daily return to this dataframe
add_daily_return_to_df(stock_a, tickers[0])
stock_a

# Delete unnamed columns in dataframe
stock_a = delete_unnamed_cols(stock_a)
stock_a

# Save cleaned dataframe to csv
save_dataframe_to_csv(stock_a, tickers[0])

## Add Daily Returns & Clean Up All Files

In [17]:
# Create a backup for all original stock data

# Cycle through all tickers
for ticker in tickers:
    print("Working on :", ticker)
    
    # Get a dataframe for that ticker
    stock_df = get_df_from_csv(ticker)
    
    # Add daily return to this dataframe
    add_daily_return_to_df(stock_df, ticker)
    
    # Delete unnamed columns in dataframe
    stock_df = delete_unnamed_cols(stock_df)
    
    # Save cleaned dataframe to csv
    save_dataframe_to_csv(stock_df, ticker)

Working on : A
Working on : AA
Working on : AAL
Working on : AAME
Working on : AAN
Working on : AAOI
Working on : AAON
Working on : AAP
Working on : AAPL
Working on : AAT
Working on : AAWW
Working on : ABBV
Working on : ABC
Working on : ABCB
Working on : ABEO
Working on : ABG
Working on : ABIO
Working on : ABM
Working on : ABMD
Working on : ABR
Working on : ABT
Working on : ABTX
Working on : AC
Working on : ACA
Working on : ACAD
Working on : ACBI
Working on : ACC
Working on : ACCO
Working on : ACER
Working on : ACGL
Working on : ACHC
Working on : ACHV
Working on : ACIW
Working on : ACLS
Working on : ACM
Working on : ACMR
Working on : ACN
Working on : ACNB
Working on : ACOR
Working on : ACRE
Working on : ACRS
Working on : ACRX
Working on : ACTG
Working on : ACU
Working on : ACY
Working on : ADBE
Working on : ADC
Working on : ADES
Working on : ADI
Working on : ADM
Working on : ADMA
Working on : ADMP
Working on : ADMS
Working on : ADNT
Working on : ADP
Working on : ADS
Working on : ADSK
W

Working on : CARA
Working on : CARG
Working on : CARS
Working on : CASA
Working on : CASH
Working on : CASI
Working on : CASS
Working on : CASY
Working on : CAT
Working on : CATB
Working on : CATC
Working on : CATO
Working on : CATY
Working on : CB
Working on : CBAN
Working on : CBAY
Working on : CBB
Working on : CBFV
Working on : CBIO
Working on : CBLI
Working on : CBOE
Working on : CBRE
Working on : CBRL
Working on : CBSH
Working on : CBT
Working on : CBTX
Working on : CBU
Working on : CBZ
Working on : CC
Working on : CCBG
Working on : CCF
Working on : CCI
Working on : CCK
Working on : CCL
Working on : CCMP
Working on : CCNE
Working on : CCO
Working on : CCOI
Working on : CCRN
Working on : CCS
Working on : CCXI
Working on : CDAY
Working on : CDE
Working on : CDEV
Working on : CDK
Working on : CDLX
Working on : CDMO
Working on : CDNA
Working on : CDNS
Working on : CDR
Working on : CDTX
Working on : CDW
Working on : CDXC
Working on : CDXS
Working on : CDZI
Working on : CE
Working on : 

Working on : ESP
Working on : ESPR
Working on : ESRT
Working on : ESS
Working on : ESSA
Working on : ESTC
Working on : ESTE
Working on : ESXB
Working on : ETH
Working on : ETN
Working on : ETR
Working on : ETRN
Working on : ETSY
Working on : EVBG
Working on : EVBN
Working on : EVC
Working on : EVER
Working on : EVFM
Working on : EVH
Working on : EVI
Working on : EVOK
Working on : EVOL
Working on : EVOP
Working on : EVR
Working on : EVRG
Working on : EVRI
Working on : EVTC
Working on : EW
Working on : EWBC
Working on : EXAS
Working on : EXC
Working on : EXEL
Working on : EXLS
Working on : EXP
Working on : EXPD
Working on : EXPE
Working on : EXPI
Working on : EXPO
Working on : EXPR
Working on : EXR
Working on : EXTN
Working on : EXTR
Working on : EYE
Working on : EYEG
Working on : EYES
Working on : EYPT
Working on : EZPW
Working on : F
Working on : FAF
Working on : FANG
Working on : FARM
Working on : FARO
Working on : FAST
Working on : FATE
Working on : FB
Working on : FBC
Working on : F

Working on : INSP
Working on : INSW
Working on : INT
Working on : INTC
Working on : INTG
Working on : INTT
Working on : INTU
Working on : INUV
Working on : INVA
Working on : INVE
Working on : INVH
Working on : IO
Working on : IONS
Working on : IOR
Working on : IOSP
Working on : IOVA
Working on : IP
Working on : IPAR
Working on : IPG
Working on : IPGP
Working on : IPI
Working on : IPWR
Working on : IQV
Working on : IR
Working on : IRBT
Working on : IRDM
Working on : IRIX
Working on : IRM
Working on : IRMD
Working on : IROQ
Working on : IRT
Working on : IRTC
Working on : IRWD
Working on : ISBC
Working on : ISDR
Working on : ISIG
Working on : ISNS
Working on : ISR
Working on : ISRG
Working on : ISSC
Working on : ISTR
Working on : IT
Working on : ITCI
Working on : ITGR
Working on : ITI
Working on : ITIC
Working on : ITRI
Working on : ITT
Working on : ITW
Working on : IVAC
Working on : IVC
Working on : IVR
Working on : IVZ
Working on : IZEA
Working on : JACK
Working on : JAGX
Working on : J

Working on : NEE
Working on : NEM
Working on : NEO
Working on : NEOG
Working on : NEON
Working on : NERV
Working on : NETE
Working on : NEU
Working on : NEWR
Working on : NFBK
Working on : NFG
Working on : NFLX
Working on : NGS
Working on : NGVC
Working on : NGVT
Working on : NHC
Working on : NHI
Working on : NHTC
Working on : NI
Working on : NICK
Working on : NINE
Working on : NJR
Working on : NKE
Working on : NKSH
Working on : NKTR
Working on : NL
Working on : NLS
Working on : NLSN
Working on : NLY
Working on : NMIH
Working on : NMRK
Working on : NNBR
Working on : NNI
Working on : NNN
Working on : NNVC
Working on : NOC
Working on : NODK
Working on : NOG
Working on : NOV
Working on : NOVT
Working on : NOW
Working on : NP
Working on : NPK
Working on : NPO
Working on : NPTN
Working on : NR
Working on : NRC
Working on : NRG
Working on : NRIM
Working on : NRZ
Working on : NSA
Working on : NSC
Working on : NSEC
Working on : NSIT
Working on : NSP
Working on : NSSC
Working on : NSTG
Working 

Working on : RRTS
Working on : RS
Working on : RSG
Working on : RUBY
Working on : RUN
Working on : RUSHA
Working on : RUSHB
Working on : RUTH
Working on : RVI
Working on : RVNC
Working on : RVP
Working on : RVSB
Working on : RWT
Working on : RXN
Working on : RYAM
Working on : RYI
Working on : RYN
Working on : RYTM
Working on : SABR
Working on : SACH
Working on : SAFE
Working on : SAFM
Working on : SAFT
Working on : SAGE
Working on : SAH
Working on : SAIA
Working on : SAIC
Working on : SAIL
Working on : SAL
Working on : SALM
Working on : SAM
Working on : SAMG
Working on : SANM
Working on : SANW
Working on : SASR
Working on : SATS
Working on : SAVE
Working on : SBAC
Working on : SBCF
Working on : SBFG
Working on : SBGI
Working on : SBH
Working on : SBNY
Working on : SBOW
Working on : SBRA
Working on : SBSI
Working on : SBT
Working on : SBUX
Working on : SC
Working on : SCHL
Working on : SCHN
Working on : SCHW
Working on : SCI
Working on : SCL
Working on : SCON
Working on : SCOR
Working o

Working on : USX
Working on : UTHR
Working on : UTI
Working on : UTL
Working on : UTMD
Working on : UUU
Working on : UVE
Working on : UVSP
Working on : UVV
Working on : V
Working on : VAC
Working on : VAL
Working on : VALU
Working on : VAPO
Working on : VBFC
Working on : VBIV
Working on : VBTX
Working on : VC
Working on : VCEL
Working on : VCRA
Working on : VCTR
Working on : VCYT
Working on : VEC
Working on : VECO
Working on : VEEV
Working on : VER
Working on : VERI
Working on : VERU
Working on : VFC
Working on : VG
Working on : VGR
Working on : VHC
Working on : VIAV
Working on : VICI
Working on : VICR
Working on : VIRC
Working on : VIRT
Working on : VISL
Working on : VIVO
Working on : VKTX
Working on : VLGEA
Working on : VLO
Working on : VLY
Working on : VMC
Working on : VMI
Working on : VMW
Working on : VNCE
Working on : VNDA
Working on : VNE
Working on : VNO
Working on : VNRX
Working on : VNTR
Working on : VOXX
Working on : VOYA
Working on : VPG
Working on : VRA
Working on : VRAY
Wo

## Get Stock Return over Time Period & Coefficient of Variation

In [45]:
stock_a

# Get total return since 2017
# Final Price 167.67 = (44.77 * 2.745) + 44.77
get_roi_defined_time(stock_a)

# Get coefficient of variation 
# This is higher than normal because I'm using many years instead of one
# get_cov(stock_a)

Initial Price : 44.77385330200195
167.6699981689453
Final Price : 167.6699981689453


1165    2.744819
Name: Adj Close, dtype: float64